# Programming and Database project

## Cars for sale in the US: an analysis





**Student**: Gianello Alessio <br>
**ID**: VR502062

## **PART 1:** Import and clean the dataset

dataset source: https://www.kaggle.com/datasets/juanmerinobermejo/us-sales-cars-dataset



Useful imports:


In [4]:
import pandas as pd

In [5]:
# Reading data
cars_list_df = pd.read_csv('cars.csv', encoding='utf-16')

In [6]:
# Taking a look at what we have at hand
cars_list_df.head(10)

,Brand,Model,Year,Status,Mileage,Dealer,Price
0,Mazda,CX-5,2023,New,NaN,NaN,36703.0
1,Kia,Sportage,2023,New,NaN,Classic Kia,28990.0
2,Chevrolet,Camaro,2024,New,NaN,Classic Chevrolet Beaumont,41425.0
3,Ford,Bronco,2023,Used,1551.0,Mike Smith Chrysler Dodge Jeep RAM,58900.0
4,Acura,TLX,2021,Used,30384.0,Mike Smith Nissan,34499.0
5,Volkswagen,Golf,2022,Certified,13895.0,Volkswagen of Beaumont,34000.0
6,GMC,Yukon,2021,Used,68506.0,BMW of Beaumont,56954.0
7,BMW,M340,2023,New,NaN,BMW of Beaumont,61715.0
8,Hyundai,Sonata,2023,New,NaN,Hyundai of Silsbee,37945.0
9,Hyundai,Sonata,2023,New,NaN,Hyundai of Silsbee,33495.0


In [7]:
# Taking a closer look
cars_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51793 entries, 0 to 51792
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Brand    51793 non-null  object 
 1   Model    51793 non-null  object 
 2   Year     51793 non-null  int64  
 3   Status   51793 non-null  object 
 4   Mileage  22981 non-null  float64
 5   Dealer   51689 non-null  object 
 6   Price    50644 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 2.8+ MB


It's evident that there are a lot of null values, mainly in column 4, more precisely:

In [14]:
cars_list_df.notnull().sum()/cars_list_df.sum()

TypeError: unsupported operand type(s) for /: 'int' and 'str'

In [8]:
cars_list_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,51793.0,2020.971425,4.215305,1959.0,2020.00,2023.0,2023.0,2024.0
Mileage,22981.0,53463.782211,44567.464826,0.0,20609.00,42158.0,73960.0,400396.0
Price,50644.0,51303.422439,35572.124619,1990.0,30983.75,45598.5,63100.5,1299995.0
